In [16]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping ,ModelCheckpoint
from sklearn.preprocessing import StandardScaler


print('lstm')

lstm


In [8]:
day=1
print('day', day)

files = glob.glob("../stock_dataset_with_label/2330/ask/*")
# print(files)

X_arr, y_arr = [], []

for f in files:
    data = pd.read_csv(f, index_col=None)
    data = data.drop(columns=['matchPri','openPri','bidPri1','bidPri2','bidPri3','bidPri4','bidPri5','askPri1','askPri2','askPri3','askPri4','askPri5'])
    X_data = data.drop(columns=['label'])
    X_data = X_data.to_numpy()
    y_data = data['label'].to_numpy()

    pastDay = day
    futureDay = 1

    for i in range(pastDay, data.shape[0]-futureDay):
        X_temp = X_data[i-pastDay : i]
        X_arr.append(X_temp)
        y_arr.append(y_data[i])
        


day 1


In [10]:
X_arr = np.array(X_arr)
y_arr = np.array(y_arr)

length = len(X_arr)
print(len(X_arr))
print(len(y_arr))

train_length = int(length*7/10)
X_train = X_arr[:train_length]
y_train = y_arr[:train_length]
X_test = X_arr[train_length:]
y_test = y_arr[train_length:]


# balanced weight 處理 平盤過多的問題 
class_weights = compute_class_weight('balanced', classes = np.unique(y_train) ,y= y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(class_weights_dict)


y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)










527057
527057
{0: 0.5998598469698004, 1: 3.003508743365137}
(368939, 1, 11) (158118, 1, 11)
(368939, 2) (158118, 2)


In [31]:
model = Sequential()

model.add(LSTM(units = 16, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.25))

model.add(LSTM(units = 16, return_sequences = True))
model.add(Dropout(0.25))

model.add(LSTM(units = 16))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))
metrics = [
        # keras.metrics.TruePositives(name='tp'),
        # keras.metrics.FalsePositives(name='fp'),
        # keras.metrics.TrueNegatives(name='tn'),
        # keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        # keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
model.compile(optimizer=SGD(learning_rate=0.001), loss='binary_crossentropy', metrics=metrics)
print(model.summary())


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 1, 16)             1792      
                                                                 
 dropout_18 (Dropout)        (None, 1, 16)             0         
                                                                 
 lstm_19 (LSTM)              (None, 1, 16)             2112      
                                                                 
 dropout_19 (Dropout)        (None, 1, 16)             0         
                                                                 
 lstm_20 (LSTM)              (None, 16)                2112      
                                                                 
 dropout_20 (Dropout)        (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 2)                

In [36]:
checkpoint =ModelCheckpoint(
    filepath="../LSTM_best.h5",
    save_weights_only=True,
    monitor='recall',
    mode='auto',
    save_best_only=True)
early_stop = EarlyStopping(monitor='recall', patience=10 ,min_delta=0.0001)
model.fit(X_train, y_train, epochs = 100, batch_size = 64, class_weight = class_weights_dict,validation_split=0.2,callbacks=[checkpoint, early_stop])
# model.fit(X_train, y_train, epochs = 100, batch_size = 32,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])


Epoch 1/100
4612/4612 [==============================] - 68s 15ms/step - loss: 0.6912 - accuracy: 0.6525 - recall: 0.6525 - auc: 0.7079 - prc: 0.6955 - val_loss: 0.6907 - val_accuracy: 0.6919 - val_recall: 0.6919 - val_auc: 0.7140 - val_prc: 0.7055
Epoch 2/100
4612/4612 [==============================] - 74s 16ms/step - loss: 0.6912 - accuracy: 0.6189 - recall: 0.6189 - auc: 0.6705 - prc: 0.6636 - val_loss: 0.6903 - val_accuracy: 0.7169 - val_recall: 0.7169 - val_auc: 0.7450 - val_prc: 0.7303
Epoch 3/100
4612/4612 [==============================] - 72s 16ms/step - loss: 0.6912 - accuracy: 0.6172 - recall: 0.6172 - auc: 0.6798 - prc: 0.6744 - val_loss: 0.6922 - val_accuracy: 0.5087 - val_recall: 0.5087 - val_auc: 0.5976 - val_prc: 0.6071
Epoch 4/100
4612/4612 [==============================] - 72s 16ms/step - loss: 0.6912 - accuracy: 0.6082 - recall: 0.6082 - auc: 0.6616 - prc: 0.6582 - val_loss: 0.6916 - val_accuracy: 0.5711 - val_recall: 0.5711 - val_auc: 0.6484 - val_prc: 0.6513
Epoc

In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score 
pred=model.predict(X_test)

4942/4942 [==============================] - 15s 3ms/step


In [38]:

print(f"Precision Score: {precision_score(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))*100:.2f}%")
print(f"Recall Score: {recall_score(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))*100:.2f}%")
print(f"F1 score: {f1_score(np.argmax(y_test, axis=1), np.argmax(pred, axis=1))*100:.2f}%")

Precision Score: 17.83%
Recall Score: 55.82%
F1 score: 27.02%
